In this notebook we are going to look through the data we are given from the other groups! (Shoutout to Davon and company)
This script isn't going to be concerned with how data is collected from marta but simply reads a JSON of one ping.

I have provided two files:

In [17]:
import os
from datetime import datetime
from dateutil import parser

cwd = os.getcwd()
print(os.listdir(cwd))

['.git', '.ipynb_checkpoints', 'busExample.json', 'busList.json', 'Feature Selection and Creation.ipynb', 'README.md', 'Untitled.ipynb']


Here we are going to visualize the two files that we have to better understand what data we are recieving and how to utilize this information

In [2]:
import json
from pprint import pprint

with open('busExample.json') as f:
    busExample = json.load(f)

The first example is one "ping" from marta's api. As you can see it is a list of timepoints for different busses.

In [3]:
pprint(busExample)

[{'adherence': '-1',
  'block_abbr': '1-1',
  'block_id': '1',
  'direction': 'Northbound',
  'latitude': '33.8191776',
  'longitude': '-84.4507974',
  'msg_time': '2019-02-18 15:25:58',
  'route': '1',
  'stop_id': '213388',
  'timepoint': 'W Marietta St & Joseph Lowery Bv',
  'trip_id': '6594976',
  'vehicle': '2506'},
 {'adherence': '0',
  'block_abbr': '1-4',
  'block_id': '4',
  'direction': 'Southbound',
  'latitude': '33.7492883',
  'longitude': '-84.4175739',
  'msg_time': '2019-02-18 15:34:25',
  'route': '1',
  'stop_id': '904948',
  'timepoint': 'Ashby Station',
  'trip_id': '6591475',
  'vehicle': '1675'},
 {'adherence': '-2',
  'block_abbr': '1-3',
  'block_id': '3',
  'direction': 'Northbound',
  'latitude': '33.7350033',
  'longitude': '-84.4139906',
  'msg_time': '2019-02-18 15:37:18',
  'route': '1',
  'stop_id': '114900',
  'timepoint': 'West End Station',
  'trip_id': '6629534',
  'vehicle': '2511'}]


For ease of visualization I am going to just print one bus' data for one time

In [4]:
instance = busExample[0]
pprint(instance)

{'adherence': '-1',
 'block_abbr': '1-1',
 'block_id': '1',
 'direction': 'Northbound',
 'latitude': '33.8191776',
 'longitude': '-84.4507974',
 'msg_time': '2019-02-18 15:25:58',
 'route': '1',
 'stop_id': '213388',
 'timepoint': 'W Marietta St & Joseph Lowery Bv',
 'trip_id': '6594976',
 'vehicle': '2506'}


In [26]:
# just game out an idea in this block of what you would like for your vector
features = []

Finally we are going to parse through the bigger file and store information about different buses:
Here we are going to find the dict of (vehicle, jsonList)

In [32]:
with open('busList.json') as f:
    busExample = json.load(f)

busDict = dict()

for entry in busExample:
    vehicle = entry['vehicle']
    if vehicle in busDict:
        busDict[vehicle] = busDict[vehicle].append(entry)
    else:
        busDict[vehicle] = [entry]
        
pprint(busDict)

{'1402': [{'adherence': '-10',
           'block_abbr': '2-2',
           'block_id': '233',
           'direction': 'Westbound',
           'latitude': '33.7737237',
           'longitude': '-84.3707827',
           'msg_time': '2019-02-18 11:27:43',
           'route': '102',
           'stop_id': '904512',
           'timepoint': 'Ponce De Leon @ Ponce City Mkt',
           'trip_id': '6625264',
           'vehicle': '1402'}],
 '1403': [{'adherence': '-4',
           'block_abbr': '126-1',
           'block_id': '107',
           'direction': 'Eastbound',
           'latitude': '33.8436796',
           'longitude': '-84.2443131',
           'msg_time': '2019-02-18 11:28:10',
           'route': '126',
           'stop_id': '210850',
           'timepoint': 'Northlake Pkwy & Parklake Dr',
           'trip_id': '6628000',
           'vehicle': '1403'}],
 '1404': [{'adherence': '-7',
           'block_abbr': '34-4',
           'block_id': '305',
           'direction': 'Southbound',
  

           'direction': 'Westbound',
           'latitude': '33.7231195',
           'longitude': '-84.5042772',
           'msg_time': '2019-02-18 11:09:44',
           'route': '71',
           'stop_id': '900177',
           'timepoint': 'Cascade Ave & Beecher St',
           'trip_id': '6623478',
           'vehicle': '1586'}],
 '1587': [{'adherence': '0',
           'block_abbr': '60-2',
           'block_id': '389',
           'direction': 'Northbound',
           'latitude': '33.7994679',
           'longitude': '-84.4755623',
           'msg_time': '2019-02-18 11:29:09',
           'route': '153',
           'stop_id': '902244',
           'timepoint': 'James Jackson Py & J Jackson Ct',
           'trip_id': '6577329',
           'vehicle': '1587'}],
 '1589': [{'adherence': '0',
           'block_abbr': '110-2',
           'block_id': '22',
           'direction': 'Northbound',
           'latitude': '33.8595591',
           'longitude': '-84.339372',
           'msg_time': '20

           'block_abbr': '81-3',
           'block_id': '460',
           'direction': 'Southbound',
           'latitude': '33.6785146',
           'longitude': '-84.4399416',
           'msg_time': '2019-02-18 11:28:29',
           'route': '81',
           'stop_id': '146900',
           'timepoint': 'East Point Station',
           'trip_id': '6577127',
           'vehicle': '1870'}],
 '1871': [{'adherence': '0',
           'block_abbr': '55-4',
           'block_id': '370',
           'direction': 'Northbound',
           'latitude': '33.7542357',
           'longitude': '-84.3923248',
           'msg_time': '2019-02-18 11:14:38',
           'route': '55',
           'stop_id': '102274',
           'timepoint': 'Alabama St & Forsyth St',
           'trip_id': '6623131',
           'vehicle': '1871'}],
 '1872': [{'adherence': '-10',
           'block_abbr': '49-1',
           'block_id': '337',
           'direction': 'Northbound',
           'latitude': '33.7538326',
           'l

Here we are going to find the dict of (route, jsonList)

In [37]:
routeDict = dict()

for entry in busExample:
    route= entry['route']
    if route in routeDict:
        temp = routeDict[route]
        temp.append(entry)
        routeDict[route] = temp
    else:
        temp = []
        temp.append(entry)
        routeDict[route] = temp
        
pprint(routeDict)

{'1': [{'adherence': '1',
        'block_abbr': '1-6',
        'block_id': '6',
        'direction': 'Northbound',
        'latitude': '33.8022399',
        'longitude': '-84.4361274',
        'msg_time': '2019-02-18 11:23:55',
        'route': '1',
        'stop_id': '213388',
        'timepoint': 'W Marietta St & Joseph Lowery Bv',
        'trip_id': '6628495',
        'vehicle': '1450'},
       {'adherence': '3',
        'block_abbr': '1-3',
        'block_id': '3',
        'direction': 'Southbound',
        'latitude': '33.7631303',
        'longitude': '-84.417528',
        'msg_time': '2019-02-18 11:24:06',
        'route': '1',
        'stop_id': '213389',
        'timepoint': 'W Marietta St & Joseph Lowery Bv',
        'trip_id': '6628496',
        'vehicle': '2511'}],
 '102': [{'adherence': '0',
          'block_abbr': '2-4',
          'block_id': '235',
          'direction': 'Eastbound',
          'latitude': '33.7755669',
          'longitude': '-84.2943392',
          'msg

          'vehicle': '1856'},
         {'adherence': '-3',
          'block_abbr': '193-3',
          'block_id': '214',
          'direction': 'Southbound',
          'latitude': '33.6745456',
          'longitude': '-84.4421111',
          'msg_time': '2019-02-18 11:26:13',
          'route': '193',
          'stop_id': '146900',
          'timepoint': 'East Point Station',
          'trip_id': '6602713',
          'vehicle': '2375'},
         {'adherence': '-3',
          'block_abbr': '193-1',
          'block_id': '212',
          'direction': 'Southbound',
          'latitude': '33.5959871',
          'longitude': '-84.3373328',
          'msg_time': '2019-02-18 11:29:36',
          'route': '193',
          'stop_id': '213208',
          'timepoint': 'Clayton State University',
          'trip_id': '6611239',
          'vehicle': '2387'}],
 '194': [{'adherence': '0',
          'block_abbr': '194-2',
          'block_id': '218',
          'direction': 'Northbound',
          'lat

         'direction': 'Eastbound',
         'latitude': '33.7559953',
         'longitude': '-84.4174209',
         'msg_time': '2019-02-18 11:24:09',
         'route': '68',
         'stop_id': '904642',
         'timepoint': 'Ashby Station',
         'trip_id': '6594003',
         'vehicle': '2509'},
        {'adherence': '0',
         'block_abbr': '68-2',
         'block_id': '397',
         'direction': 'Eastbound',
         'latitude': '33.7376649',
         'longitude': '-84.4344708',
         'msg_time': '2019-02-18 11:28:23',
         'route': '68',
         'stop_id': '900239',
         'timepoint': 'Westwood Ave & Cascade Ave',
         'trip_id': '6623320',
         'vehicle': '1680'}],
 '71': [{'adherence': '-6',
         'block_abbr': '71-4',
         'block_id': '407',
         'direction': 'Westbound',
         'latitude': '33.7231195',
         'longitude': '-84.5042772',
         'msg_time': '2019-02-18 11:09:44',
         'route': '71',
         'stop_id': '900177',


In [ ]:
dt = parser.parse(instance['msg_time'])
print(dt.date)
print(dt.time)

In [36]:
import json

def build_data_struct(json_input):
    
    with open(json_input) as f:
        dictList = json.load(f)
        
    dayDict = dict()
    
    for entry in dictList:
        vehicle = entry['vehicle']
        dt = parser.parse(entry['msg_time'])
        date = dt.date()
        if date in dayDict:
            if vehicle in dayDict[date]:
                dayDict[date][vehicle] = dayDict[date][vehicle].append(entry)
            else:
                dayDict[date][vehicle] = [entry]
        else:
            temp = dict()
            temp[vehicle] = [entry]
            dayDict[date] = temp
    return dayDict
            

In [38]:
dayDict = build_data_struct('busList.json')

In [41]:
pprint(dayDict.keys())
pprint(dayDict[dt.date()])

dict_keys([datetime.date(2019, 2, 18)])
{'1402': [{'adherence': '-10',
           'block_abbr': '2-2',
           'block_id': '233',
           'direction': 'Westbound',
           'latitude': '33.7737237',
           'longitude': '-84.3707827',
           'msg_time': '2019-02-18 11:27:43',
           'route': '102',
           'stop_id': '904512',
           'timepoint': 'Ponce De Leon @ Ponce City Mkt',
           'trip_id': '6625264',
           'vehicle': '1402'}],
 '1403': [{'adherence': '-4',
           'block_abbr': '126-1',
           'block_id': '107',
           'direction': 'Eastbound',
           'latitude': '33.8436796',
           'longitude': '-84.2443131',
           'msg_time': '2019-02-18 11:28:10',
           'route': '126',
           'stop_id': '210850',
           'timepoint': 'Northlake Pkwy & Parklake Dr',
           'trip_id': '6628000',
           'vehicle': '1403'}],
 '1404': [{'adherence': '-7',
           'block_abbr': '34-4',
           'block_id': '305',


           'route': '5',
           'stop_id': '902868',
           'timepoint': 'Hammond Dr & Barfield Rd',
           'trip_id': '6627356',
           'vehicle': '1661'}],
 '1663': [{'adherence': '0',
           'block_abbr': '5-6',
           'block_id': '351',
           'direction': 'Southbound',
           'latitude': '33.8227635',
           'longitude': '-84.3695749',
           'msg_time': '2019-02-18 11:23:32',
           'route': '5',
           'stop_id': '902145',
           'timepoint': 'Lindbergh Ctr Station (N Loop)',
           'trip_id': '6621091',
           'vehicle': '1663'}],
 '1665': [{'adherence': '-2',
           'block_abbr': '60-4',
           'block_id': '391',
           'direction': 'Southbound',
           'latitude': '33.7559326',
           'longitude': '-84.4687645',
           'msg_time': '2019-02-18 11:24:39',
           'route': '60',
           'stop_id': '903643',
           'timepoint': 'James Jackson Py & Hightower Rd',
           'trip_id': '65

           'block_abbr': '1-3',
           'block_id': '3',
           'direction': 'Southbound',
           'latitude': '33.7631303',
           'longitude': '-84.417528',
           'msg_time': '2019-02-18 11:24:06',
           'route': '1',
           'stop_id': '213389',
           'timepoint': 'W Marietta St & Joseph Lowery Bv',
           'trip_id': '6628496',
           'vehicle': '2511'}],
 '2512': [{'adherence': '-1',
           'block_abbr': '26-3',
           'block_id': '275',
           'direction': 'Eastbound',
           'latitude': '33.7852391',
           'longitude': '-84.42257',
           'msg_time': '2019-02-18 11:26:52',
           'route': '26',
           'stop_id': '904519',
           'timepoint': 'Bankhead Station',
           'trip_id': '6611965',
           'vehicle': '2512'}],
 '2513': [{'adherence': '2',
           'block_abbr': '856-3',
           'block_id': '528',
           'direction': 'Eastbound',
           'latitude': '33.7542535',
           'lon

In [37]:
def K_previous_vel (input_dict):
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-37-25871e11bdb9>, line 3)